#License and Attribution

This notebook was developed by Emilio Serrano, Full Professor at the Department of Artificial Intelligence, Universidad Politécnica de Madrid (UPM), for educational purposes in UPM courses. Personal website: https://emilioserrano.faculty.bio/

📘 License: Creative Commons Attribution-NonCommercial-ShareAlike (CC BY-NC-SA)

You are free to: (1) Share — copy and redistribute the material in any medium or format; (2) Adapt — remix, transform, and build upon the material.

Under the following terms: (1) Attribution — You must give appropriate credit, provide a link to the license, and indicate if changes were made; (2) NonCommercial — You may not use the material for commercial purposes; (3) ShareAlike — If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.

🔗 License details: https://creativecommons.org/licenses/by-nc-sa/4.0/

# Advanced LangChain Workflows: Summarization, RAG, Memory, and Web Chatbots

In this notebook, we will build more complex generative AI systems, assuming a basic understanding of LangChain.

Learning Objectives:

*  Summarize large documents that exceed the context window size

* Build a Retrieval-Augmented Generation (RAG) system to answer questions based on multiple documents.

* Manage conversation history with structured memory using LangGraph.

* Build an interactive web-based chatbot with Gradio.

# Initial Setup

First, let's install the libraries

In [3]:
# Install all necessary libraries for the notebook
!pip -q install langchain langchain-community langchain_huggingface langchain_groq langgraph  ddgs faiss-cpu beautifulsoup4


In [4]:
import sys

# This forces the shell (!) to use the kernel's Python executable (sys.executable)
!{sys.executable} -m pip show langchain

Name: langchain
Version: 1.0.3
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/federicosvendsen/Documents/UPM/DeepLearning4NLP/1-7/script/.conda/lib/python3.11/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


Configure the Groq API key.  

In [5]:
import os
from dotenv import load_dotenv

#Using google.colab secrets
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    print("🛑 Groq API Key not found. Please make sure to set it up.")
else:
    print("✅ Groq API Key configured.")

✅ Groq API Key configured.


#Summarizing Long Documents with LangChain


##Testing the model context window

Here we will check if our model can summarize a long text. [Groq models](https://console.groq.com/docs/models) specify the size of the context window for each model.

We will use LangChain WebBaseLoader to download the wikipedia page of the History of Artificial Intelligence.

⚠️ Alert: Context overflow wanted. Error "Request too large for model `meta-llama/llama-4-scout-17b-16e-instruct". If not, increase the text_excerpt length.



In [8]:
from langchain_classic.document_loaders import WebBaseLoader
from langchain_classic.prompts import PromptTemplate
import re
from langchain_groq import ChatGroq

# 1. Load article
url = "https://en.wikipedia.org/wiki/History_of_artificial_intelligence"
loader = WebBaseLoader(url)
docs = loader.load()

# 2. Combine content
raw_text = "\n\n".join([doc.page_content for doc in docs])
print(f"Original text has {len(raw_text)} characters.")

# 3. Clean up excessive line breaks
clean_text = re.sub(r'\n{2,}', '\n', raw_text)
# Optional: trim each line
clean_text = "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])
text_excerpt = clean_text
print(f"Text excerpt has {len(text_excerpt)} characters.")

# 4. Show a preview
print("\n--- Cleaned Text Sample ---")
print(text_excerpt[5000:5500])

# 5. Load LLM
llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.2, groq_api_key=api_key)
print("Groq LLM loaded.")

# 6. Summarize
print("\n--- Invoking the LLM for summarizing (error expected) ---")
response = llm.invoke(f"Summarize the following text:\n{text_excerpt}")

# 7. Show summary
print("\n--- LLM Summary ---")
print(response.content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Original text has 142018 characters.
Text excerpt has 140535 characters.

--- Cleaned Text Sample ---

Quantum
By country
Bulgaria
Eastern Bloc
Poland
Romania
South America
Soviet Union
Yugoslavia
Timeline of computing
before 1950
1950–1979
1980–1989
1990–1999
2000–2009
2010–2019
2020–present
more timelines ...
Glossary of computer science
Categoryvte
The history of artificial intelligence (AI) began in antiquity, with myths, stories, and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The study of logic and formal reasoning from antiquity to the prese
Groq LLM loaded.

--- Invoking the LLM for summarizing (error expected) ---


APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01k5pqw61sf97t06kkar2fk64p` service tier `on_demand` on tokens per minute (TPM): Limit 30000, Requested 34842, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

##Map-reduce approach

When dealing with text that is too long to fit into a single API call, a common strategy is to use a `Map-Reduce` approach. This involves:

* Splitting the large document into smaller, more manageable chunks.

* Mapping by generating a summary for each individual chunk.

* Reducing the results by combining all the smaller summaries into one final, concise summary.

`load_summarize_chain`is a function provided by LangChain that creates a summarization pipeline. It accepts an LLM and a `chain_type` parameter.  `chain_type="map_reduce"` indicates to summarize each chunk individually ("map") and then combine the results into a final summary ("reduce").

`RecursiveCharacterTextSplitter` will be used to break long documents into manageable chunks. `RecursiveCharacterTextSplitter` does this intelligently by attempting to split at natural boundaries (such as paragraphs or sentences), while also allowing for **overlap between chunks** to preserve context across segments. For example, with a `chunk_size` of 1000 characters and a `chunk_overlap` of 200, the splitter ensures that each new chunk shares 200 characters with the previous one.  


⚠️ Alert: the code may exceeds token-per-minute (TPM) limit. This is not a context window overflow. Groq's service tier (on_demand) restricts the number of tokens you can send per minute. Check [Groq Rate Limits](https://console.groq.com/docs/rate-limits). For instance, meta-llama/llama-4-scout-17b-16e-instruct has 30K TPM vs llama-3.1-8b-instant with 6K TPM. Note that 1 token ≈ 3.5–4 average English characters.


In [11]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 153.5 kB/s  0:01:17m0:00:0200:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers] [transformers]


In [13]:
from langchain_groq import ChatGroq
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_classic.chains.summarize import load_summarize_chain

# Step 1: Loading a LLM model
llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.2, groq_api_key=api_key)
print("Groq LLM loaded successfully.")

# Step 2: Prepare the Long Text
long_text=text_excerpt

print(f"The text has {len(long_text)} characters.")

# Step 3: Split the Text into Chunks
# We'll use RecursiveCharacterTextSplitter from LangChain. It's the recommended tool for this task as it intelligently splits text while trying to keep paragraphs and sentences intact.

# Create an instance of the splitter
# Fit the chunk_size to the context window of LLL, the larger and more permissible the better
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=9000,  # Maximum size of each chunk in characters
    chunk_overlap=500, # Overlapping characters to maintain context between chunks
    length_function=len # Function used to measure the length of each chunk (default: character count)
)

# Split the text. The output is a list of strings.
text_chunks = text_splitter.split_text(long_text)

# To use the LangChain chain, we convert the strings into Document objects.
docs = [Document(page_content=t) for t in text_chunks]

print(f"The text has been divided into {len(docs)} documents.")
#print("First chunk:", docs[0].page_content)

# Step 4: Create and Execute the Map-Reduce Chain
# LangChain makes this easy with the `load_summarize_chain` function. We specify the `map_reduce` or `refine` chain type.
# LangChain will use default prompts, but we can also define custom prompts for more control. For this example, we'll use the optimized default prompts.
# Create the chain, specifying the LLM and the strategy type.
# `verbose=True` can show us the steps LangChain is performing behind the scenes.
map_reduce_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    verbose=False
)

# Run the chain on our documents!
result = map_reduce_chain.invoke({"input_documents": docs})

# Step 5: View the Final Result
# Finally, we print the generated summary.
print("\n--- Final Generated Summary ---\n")
print(result["output_text"])

# You have now successfully implemented a Map-Reduce summarization pipeline. The verbose output showed you how the LLMChain was first run on each chunk (the Map phase),
# and then a different chain was used to combine those intermediate summaries (the Reduce phase) into the final result.

Groq LLM loaded successfully.
The text has 140535 characters.
The text has been divided into 17 documents.


ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.

##Refine approach
Another powerful technique is the `Refine` method. This approach works differently:

* It starts by generating an initial summary of the first chunk of text.

* It then iteratively refines this summary by taking the next chunk and the existing summary, and asking the LLM to update the summary with the new information.

* This process continues until all chunks have been processed.

The `Refine` method is particularly effective when the final summary needs to be highly detailed and coherent, as it builds on previous context. However, because it is a sequential process, it is often slower than the Map-Reduce approach.

In the previous code, you only need to change  `chain_type` parameter of the `load_summarize_chain` function that returns the summarization pipeline.

Let us see if we get a better summary.

In [ ]:
# Steps 1-3: steps 1 to 3 equal to the map reduce code

# Step 4: Create and execute a Refine summarization chain
# The "refine" chain summarizes progressively by improving the summary with each new chunk
refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=False  # Set to True to see step-by-step execution
)

# Run the refine summarization chain
result = refine_chain.invoke({"input_documents": docs})

# Step 5: Output the final summary
print("\n--- Final Generated Summary (Refine Chain) ---\n")
print(result["output_text"])

## Customizing the summary
You  can  customize both map_reduce and refine chains in LangChain by providing your own prompt templates. This is useful when you want more control over:
* The tone, detail, or style of the summaries
* Instructions for how to summarize
* Contextual awareness in multi-stage summarization

Let us repeat the refine summary... as a pirate.

In [ ]:
# Steps 1-3: steps 1 to 3 equal to the map reduce code

initial_prompt = PromptTemplate(
    input_variables=["text"],
    template="Write an initial summary of the following content AS A PIRATE:\n\n{text}"
)

refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template="Here is the current summary:\n\n{existing_answer}\n\nRefine it using the additional text below AS A PIRATE:\n\n{text}"
)

refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    verbose=False
)

result = refine_chain.invoke({"input_documents": docs})

print("\n--- Final Generated Summary (Refine Chain) ---\n")
print(result["output_text"])

##Beyond Summarization
LangChain offers a broader set of chain pipelines beyond `load_summarize_chain`, many of which support Map-Reduce, Refine, and other strategies to process long texts that exceed an LLM’s context window.

These pipelines cover a wide range of use cases, including question answering (QA), retrieval-based generation, structured reasoning, and multi-prompt workflows.

For example, `load_qa_chain` enables question answering over large document sets and supports several chain types:

* "stuff" – Concatenates all documents into a single prompt (best for small inputs)

* "map_reduce" – Answers each chunk independently, then summarizes the results

* "refine" – Generates an initial answer and iteratively refines it with new chunks

* "map_rerank" – Generates multiple answers per chunk, scores them, and selects the bes



#RAG: Answering Questions with External Documents

In this section, we will build the a **Retrieval-Augmented Generation (RAG)** system. The goal is to enable a language model (LLM) to answer questions based on external documents — especially useful when those documents contain up-to-date or domain-specific knowledge not included in the model’s pretraining.




##Testing the recent fatual knowledge of a LLM

In this section, we will test how much our Language Model (LLM) knows about a recent or updated topic. This will help us understand the limitations of the model's internal knowledge when it comes to current events or newly emerged facts.

We’ll be using the `llama-3.1-8b-instant model`, which was pretrained before 2025. This means it should not have direct knowledge of events or information that occurred after its cutoff date.

To ensure accurate and consistent results for factual queries, we’ll also set a low temperature.



In [ ]:
from langchain_groq import ChatGroq
#Let us load a LLM from groq
# Let us select a low temperature, good for tasks requiring consistency and precision as fact-based tasks
llm_low_temp = ChatGroq(model_name="llama-3.1-8b-instant",temperature=0.2, groq_api_key=api_key)

response_NO_RAG = llm_low_temp.invoke("¿Cuántos parámetros tiene el modelo DeepSeek?")
print("\n--- LLM Answer without RAG contex ---")
print(response_NO_RAG.content)

## Document Loading and Splitting Pipeline  

This code performs two main tasks:

* It loads web content from specified URLs using LangChain’s `WebBaseLoader`.
* It splits the content into smaller, manageable chunks using a `RecursiveCharacterTextSplitter`, making it suitable for LLM input.

`WebBaseLoader` is a utility in LangChain that allows you to automatically fetch and extract content from web pages (HTML). It simplifies the process of turning online information into a format that LLMs can use.

Since LLMs have a limit on the amount of text they can process at once, long documents must be split into smaller chunks. The `RecursiveCharacterTextSplitter` does this intelligently by attempting to split at natural boundaries (such as paragraphs or sentences), while also allowing for **overlap between chunks** to preserve context across segments. For example, with a `chunk_size` of 1000 characters and a `chunk_overlap` of 200, the splitter ensures that each new chunk shares 200 characters with the previous one.  


Note: In this notebook, we illustrate RAG using a fixed list of websites loaded with WebBaseLoader, which is ideal for loading specific internal, private, or curated documents. However, apps like [Tavily](https://www.tavily.com/), a dynamic web search API designed for LLMs, can be seamlessly integrated to enable the system to autonomously discover relevant and up-to-date content from the web.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# --- 1. Load Documents (Loader) ---
# We use a web loader to fetch content from multiple authoritative URLs.
print("Loading documents from the web...")
loader = WebBaseLoader([
     "https://en.wikipedia.org/wiki/DeepSeek", #not that it is in English
     "https://en.wikipedia.org/wiki/DeepSeek_(chatbot)",
     # Add more  URLs here if needed
])

# Load documents from the specified URLs
docs = loader.load()
print(f"Loaded {len(docs)} document(s).")

# Optional: Clean Up Excessive Line Breaks in Page Content ---
def clean_text(text):
    # Remove multiple line breaks and strip trailing spaces
    text = re.sub(r'\n{2,}', '\n', text)
    text = '\n'.join(line.strip() for line in text.splitlines() if line.strip())
    return text

for doc in docs:
    doc.page_content = clean_text(doc.page_content)

# --- 2. Split Text (Splitter) ---
# We split each document into smaller chunks so the model can process them effectively.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
#Each element in splits is a Document with a page_content attribute that contains the text of that fragment.

# Confirm number of splits
print(f"Split into {len(splits)} chunks (chunk_size=1000, overlap=200).")

print("\n--- Content of 10th chunk ---")
print(splits[10].page_content)


##RAG Pipeline: Creating Embeddings and Vector Store

We will perform three main steps:

1. **Create vector embeddings**  
   We transform each text chunk into a numerical vector using a **multilingual sentence embedding model** from Hugging Face. This captures the semantic meaning of each chunk, allowing for accurate similarity comparisons.  We're using `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`, which supports multiple languages — ideal for multilingual document sets.    There are many alternatives depending on your needs (e.g., English-only, faster, domain-specific).

2. **Store vectors in a FAISS database**  
   [FAISS](https://faiss.ai/) (Facebook AI Similarity Search) is a high-performance library for fast vector similarity search. It allows us to quickly find the most relevant chunks for a given user query. You could replace FAISS with other vector stores such as Chroma, Qdrant, Weaviate, or Pinecone — especially in production use cases.

3. **Create a RAG chain**  
   We combine the retriever with a prompt-driven LLM chain.

This architecture enhances factual accuracy and reduces hallucination by grounding the model’s answers in verifiable sources.

In [ ]:

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# --- 3. Create Embeddings and Vector Store ---
# We convert the text chunks into dense vectors (embeddings) using a multilingual model.
# These vectors capture the semantic meaning of the text for similarity search.
print("Creating vector database...")
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
# Note: You can replace this model with another multilingual or domain-specific one if needed.

# FAISS stores the embeddings and supports fast vector similarity search.
# Alternatives include Chroma, Qdrant, Weaviate, Pinecone, etc.
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings_model)

# --- 4. Create the Retriever ---
# This component queries the vector store and returns the most relevant text chunks.
retriever = vectorstore.as_retriever()

# --- 5. Create the RAG Chain ---
print("Building the RAG chain...")

# This prompt defines the system's behavior and instructs it to use the retrieved context.
# The {context} placeholder will be automatically filled by LangChain
system_prompt = """
You are an expert assistant for answering questions.
Use the following retrieved context to answer the user's question.
If you don't know the answer based on the context, just say so.

Context:
{context}
"""

# Define a prompt template that includes both the system instruction and the user's question.
# {input} will be filled when asking the question
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

# This document chain takes the retrieved documents and formats them for the LLM.
# This sub-chain injects the context + user input into the LLM using the above prompt.
rag_prompt_chain = create_stuff_documents_chain(llm_low_temp, prompt_template)

# The final RAG chain combines retrieval + generation.
# It retrieves documents and passes them to the LLM with the prompt.
rag_chain = create_retrieval_chain(retriever, rag_prompt_chain)




## Querying the RAG System

Now that we've built the full RAG pipeline — including document embeddings, retrieval, and prompt formatting — we can test it by asking a question.

In this step:

- We send a natural language query to the `rag_chain`.
- The system uses the retriever to find relevant document chunks based on semantic similarity.
- Those chunks are injected into the prompt as context.
- The language model then generates an answer based only on that context.
- If the context does not contain the necessary information, the model should respond with "I don't know".

Note that the retriever returns several document chunks that are combined and passed as context for the LLM. *Passing too many or large chunks can exceed the model’s token limit*. To prevent this, it’s common to limit the number of chunks retrieved, split documents into smaller pieces, or use more advanced chains like `map_reduce` that handle large contexts iteratively.



In [ ]:
# --- 6. Ask a Question ---
question = "¿Cuántos parámetros tiene el modelo DeepSeek?"
print(f"\nQuestion: {question}")

response = rag_chain.invoke({"input": question})

print("\n--- RAG System Answer ---")
print(response["answer"])

# The Memory Challenge: Building a Conversational Chat

A crucial concept to understand is that LLMs are stateless. By default, every time you send a request (`invoke`), the model treats it as a completely new, independent interaction. It has no memory of your previous questions or its own previous answers.

This is a problem for building chatbots. Let's demonstrate this.

##The Problem: A Stateless Conversation
We will tell the model our name and its name in one request. Then, in a separate request, we'll ask it to recall the information.



In [ ]:
# First interaction: We provide names
print("--- Interaction 1: Providing Information ---")
first_prompt = "Hi! My name is Emilio, and I'll call you Smart-Bot."
first_response = llm.invoke(first_prompt)
print(f"Me: {first_prompt}")
print(f"LLM: {first_response.content}")

# Second, separate interaction: We ask it to recall the information
print("\n--- Interaction 2: Asking for Recall ---")
second_prompt = "What is my name?"
second_response = llm.invoke(second_prompt)
print(f"Me: {second_prompt}")
print(f"LLM: {second_response.content}")

As you can see, the LLM has no idea what our name is. The second call was a completely separate transaction with no context from the first one.

## The Solution: Managing Chat History

To have a conversation, we must manually include the history of the conversation in every new request. We provide the previous turns of dialogue as context for the model to use.

LangChain uses a specific message format for this (`HumanMessage`, `AIMessage`). Let's manage the history ourselves to see how it works under the hood.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# We will store the conversation history in a simple list
chat_history = []

# --- Interaction 1 ---
print("--- Interaction 1 (with history) ---")
prompt_1 = HumanMessage(content="Hi! My name is Emilio, and I'll call you Smart-Bot.")
response_1 = llm.invoke([prompt_1]) # We send the message inside a list

# Add the first exchange to our history
chat_history.append(prompt_1)
chat_history.append(response_1)

print(f"Me: {prompt_1.content}")
print(f"LLM: {response_1.content}")


# --- Interaction 2 ---
print("\n--- Interaction 2 (with history) ---")
prompt_2 = HumanMessage(content="Great! Now, do you remember what my name is?")

# IMPORTANT: We include the previous history in our new request!
# The LLM now has the context it needs.
response_2 = llm.invoke(chat_history + [prompt_2])

# We could continue adding this new exchange to the history if the chat were to continue
# chat_history.append(prompt_2)
# chat_history.append(response_2)

print(f"Me: {prompt_2.content}")
print(f"LLM: {response_2.content}")


Success! By passing the previous messages along with our new question, the model had the necessary context to answer correctly.

This code demonstrates the fundamental principle of conversational memory. For more complex applications, the current best practice is to use LangGraph, which replaces LangChain’s previous memory components and offers a more robust and scalable system for managing dialogue history

##  Building an Interactive Chatbot with Memory and a GUI


The previous example showed why explicit memory management—manually passing a list of messages—is necessary but can quickly get complicated. Now, let’s build a much more elegant and powerful chatbot that feels like a real application.

This chatbot will:


- Employ LangGraph’s `StateGraph` as memory backend, which manages conversation states in a graph structure. This allows handling complex dialogue flows, including cycles and state transitions, more naturally than simple lists.

- Leverage `MessagesState`, a  state schema designed for managing message histories within LangGraph.

- Define a clear System Role to give the assistant a distinct personality and rules as done in previous examples.

- Include a  Graphical User Interface (GUI) built with Gradio, enabling direct interaction through a browser.

`LangGraph` complements LangChain by adding orchestration capabilities and fine-grained control over conversation state using graph-based logic. While LangChain provides the foundational components—such as prompt templates, chains, and tool integration—LangGraph focuses on modeling dialogue as dynamic state transitions. This separation of concerns allows for more scalable and maintainable conversational agents.

To build the GUI, we'll use `Gradio`. Gradio is a  Python library that allows us to create and share a web-based user interface for our machine learning models with just a few lines of code.


In [16]:
# 1. Install necessary libraries
# We need gradio for the GUI and langgraph for the modern conversational flow.
!pip install langchain langchain-groq gradio langgraph -q

# 2. Import all required modules
import os
import gradio as gr
from typing import List

# --- LangChain & LangGraph Core Data Structures ---
# BaseMessage: The abstract base class for all message types. Every turn in a conversation is a `BaseMessage`.
# HumanMessage: Represents a message from the user.
# AIMessage: Represents a message from the AI model.
# SystemMessage: A message that sets the context and instructions for the AI, not part of the back-and-forth conversation.
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq

# StateGraph: The main class for building stateful, potentially cyclical graphs.
# END: A special node that signifies the end of a graph's execution.
from langgraph.graph import StateGraph, END

# MessagesState: A pre-built helper from LangGraph for managing state.
# It's essentially a TypedDict with one key: `messages`, which is a list of `BaseMessage` objects.
# Its structure is: {"messages": [BaseMessage, HumanMessage, AIMessage, ...]}
from langgraph.graph.message import MessagesState

Let us create *improBot — Your Instant Comedy Sketch Creator*.

A witty chatbot that crafts original, hilarious comedy sketches by blending all the ideas you’ve shared throughout the conversation — no questions asked, just pure improv humor.

This task is quite challenging—even for a human—but it serves as a way to evaluate the memory capabilities of our chatbot.

Note: After multiple unsuccessful attempts using the LLaMA 3 8B model, I switched to the more powerful LLaMA 3 70B.


In [17]:
# 3. Configure the Groq LLM
try:
    load_dotenv()
    #Using google.colab secrets
    api_key = os.getenv("GROQ_API_KEY")
    os.environ['GROQ_API_KEY'] = api_key
    print("✅ Groq API Key configured.")
except Exception as e:
    print(f"🛑 Error getting API Key: {e}")
    print("Please configure the 'GROQ_API_KEY' secret in Google Colab.")

# 4. Initialize the language model and chatbot personality
#let us use a bigger and more powerful model!
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
# The bot's personality is defined here as a `SystemMessage`.
# This message will be added to the beginning of the conversation history on every single call.
system_prompt_text = """

You are improBot, a creative comedy sketch writer with a sharp sense of humor
and an uncanny ability to weave diverse ideas into hilarious, seamless sketches.
Your task is to write original comedic sketches that incorporate all relevant elements
and ideas the user has provided during the entire conversation.
You never ask questions or request clarifications.
Instead, you use the full conversation history as your sole inspiration to craft each new sketch from scratch.
Do NOT ignore earlier parts of the conversation.
Use every piece of information shared before to build a funny, engaging sketch that entertains and surprises.

"""
system_message = SystemMessage(content=system_prompt_text)

# 5. Define the Conversation Graph using LangGraph

# NOTE: This is a minimal linear graph with a single node and no branching or cycles.
# It's a great starting point to understand how LangGraph works before adding complexity.

# Define the graph's node: a function that will call our LLM
#    It takes the current state (`MessagesState` dict) as input
#    This dictionary has the structure {"messages": [BaseMessage, HumanMessage, AIMessage, ...]}
def call_model(state: MessagesState):
    """Invokes the LLM with the current list of messages."""
    # The `state` dictionary contains the key 'messages', which holds the conversation history.
    response = llm.invoke(state['messages'])
    # We return a dictionary that matches the `MessagesState` structure.
    # The graph will automatically append this new AI message to the 'messages' list.
    return {"messages": [response]}

# Instantiate the StateGraph. We define the "shape" of our state using `MessagesState`.
# This tells the graph that its state will always be a dictionary with a 'messages' key.
workflow = StateGraph(MessagesState)

# Add our defined function call_model as a node named "llm" to the graph.
workflow.add_node("llm", call_model)

# Set the entry point of the graph. Execution will always start at the "llm" node.
workflow.set_entry_point("llm")

# Set the end point. After the "llm" node runs, the graph execution finishes.
workflow.add_edge("llm", END)

# Compile the workflow into a runnable application.
runnableApp= workflow.compile()

print("LangGraph graph compiled.")


# 6. Create the function to be called by the Gradio GUI
def myChatbot_langgraph(user_message: str, history: List[List[str]]):
    """
    This function bridges Gradio with our LangGraph application.
    1. Converts Gradio's simple list-based history into LangChain's structured message format.
    2. Invokes the LangGraph app with the complete conversation state.
    3. Returns the AI's response string to the Gradio UI.
    """
    # Gradio's `history` format is a simple list of lists: [["user input", "ai response"], ...].
    # We need to convert this into the `List[BaseMessage]` format LangChain/LangGraph expects.
    langchain_messages = [system_message] # Always start with the system prompt with the chatbot personality
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))

    # Add the user's latest message (parameter of myChatbot_langgraph that will be called by the Gradio Interface)
    langchain_messages.append(HumanMessage(content=user_message))

    # Invoke the graph with a state dictionary that matches the `MessagesState` schema.
    response_state = runnableApp.invoke({"messages": langchain_messages})

    # The `response_state` is the final state of the graph. Its 'messages' list contains the entire conversation,
    # with the very last message being the new AI response (index -1 in a Python List).
    return response_state['messages'][-1].content


# 6. Launch the Gradio Chat Interface
print("\n🚀 Launching the chat interface (LangGraph version)...")
gr.ChatInterface(
    myChatbot_langgraph,
    title="improBot — Your Instant Comedy Sketch Creator",
    description="A witty chatbot that crafts original, hilarious comedy sketches by blending all the ideas you’ve shared throughout the conversation — no questions asked, just pure improv humor",
    chatbot=gr.Chatbot(height=400),
    theme="soft"
).launch(debug=True, share=True)

✅ Groq API Key configured.
LangGraph graph compiled.

🚀 Launching the chat interface (LangGraph version)...


/var/folders/t2/d3hrzr8x7zd9cnztszgjcw0h0000gn/T/ipykernel_13750/425621199.py:98: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=400),
/Users/federicosvendsen/Documents/UPM/DeepLearning4NLP/1-7/script/.conda/lib/python3.11/site-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/11/06 13:14:29 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None


# Conclusions and Future Steps

In this notebook, we explored  advanced architectures for building generative AI systems:

* Summarizing Long Documents with LangChain by using the Map-reduce and Refine approaches.

* Retrieval-Augmented Generation (RAG): We saw how RAG systems enable language models to retrieve and synthesize information from external document collections, improving factual accuracy and grounding responses.

*  Manage conversation history with structured memory using LangGraph and bulding  an interactive web-based chatbot with Gradio.

**Next Steps**

* Deploy your own chatbot prototype for free using [Gradio on Hugging Face Spaces](https://www.gradio.app/main/guides/deploying-gradio-with-docker).

* Build a multi-agent LLM system with LangGraph or AgentCrew:   Build networks of agents that collaborate, challenge, or specialize in different roles.

